<div style="font-family:verdana">
<h2>Table of Content</h2>
<ol style="font-size:16px;list-style-type:none">
    <li><a href="#plot">Beyond the Plot Use Case</a></li>
    <li><a href="#why-gemini">Why Gemini 1.5</a></li>
    <li><a href="#context-caching">Context Caching</a></li>
    <li><a href="#install-libraries">Install and Import Libraries</a></li>
    <li><a href="#configure-gemini">Configure gemini API</a></li>
    <li><a href="#extract-text">Extract text from books</a></li>
    <li><a href="#write-prompts">Write required prompts</a></li>
    <li><a href="#create-context-caching">Creating context cache</a></li>
    <li><a href="#prizoner-of-azkaban">Harry Potter and Prizoner of Azkaban</a></li>
    <li><a href="#delete-cache-1">Delete Cache of Prizoner of Azkaban</a></li>
    <li><a href="#bond-return">Bond return in Order of Phoenix</a></li>
    <li><a href="#delete-cache-2">Delete Cache of Order of Phoenix</a></li>
     <li><a href="#conclusion">Conclusion</a></li>
    
</ol>
</div>

<div style="font-family:verdana"  id="plot">
<h1 >Beyond the Plot Use Case</h1>
<p style="font-size:16px">This use case aims to creatively alter the storyline of Harry Potter books by introducing James Bond, the iconic British spy, into the magical universe. Using Google Gemini-1.5-flash model, we can generates rewritten content, new character dialogues, and alternate story arcs and much more creative things. It showcases how <b style="color:blue"> long context window </b> capability of Gemini 1.5 models can enhance literary creativity and provide a fresh perspective on classic stories/novels. </p>

<p style="font-size: 16px">We are going to introduce James Bond in the 3rd and 5th edition of Harry Potter series. 3rd edition contains more than 150K tokens and 5th edition contains more than 350K tokens. In use case there will be multiple prompts on same input so we are using <b style="color:blue"> context caching </b> to demonstrate how much efficiently and cost effectively we can use gemini 1.5 models long context window</p>

<p style="font-size: 16px">Specially in this use case where we can do many creating things on large document. To process such a large token size at once we need powerfull models like gemini-1.5-flash, gemini-1.5-pro to hold the context of complete document and get more contextual understanding.</p>

<p style="font-size: 16px">We will see more details of the use case as we move further</p>
</div>

<div style="font-family:verdana" id="why-gemini">
    <h2 >Why Gemini-1.5 models (we are using gemini-1.5-flash-002) over other models</h2>
    <p style="size:16px">Let's take a look at certain prominent models context window : gemini 1.0 has 32k, gpt 4 turbo has 128k, claude 2.1 has 200k, gemini-1.5-flash has 1 million and gemini-1.5-pro has 2 million
    </p>
    <p style="size:16px">In our use case we are firstly working on Harry Potter and Prizoner of Azkaban book and giving some 
    interesting prompts to make model think in more creative way. We will ask model the dialogues of new character, summary of 
    the book after incoporating the new character and more. This book has more than 150k tokens. Then after taking summary of this 3rd edition book then we will feed 5th edition Harry Potter and Order of Phoenix which has more than 350k tokens and adding summary of altered version of 3rd edition. So currently only gemini 1.5 models can give best results to our use case, mainly because of its long context ability</p>
</div>

<div style="font-family:verdana" id="context-caching">
    <h2>Context Caching</h2>
    <p style="size:16px">In our use case, we are giving prompts multiple times to the same input tokens, so to work with them efficiently and cost effectively we are using context caching. If we work without caching then if we have 3 quueries(prompts) then at all 3 requests the input tokens will get loaded by the model. This will generate more costs and will take more time to execute as each time we are ingesting input to the model. With the context cache, we cache the input tokens (context window) first and then give all the required prompts which we need to do perfom on the input. The number of input tokens cached, billed at a reduced rate when included in subsequent prompts. The amount of time cached tokens are stored at specific time which we mention in TTL and cache is billed hourly.</p>
    <p style="size:16px">We are using gemini-1.5-flash model for our use case which can store 1 million context cache</p>
</div>

<div style="font-family:verdana" id="install-libraries">
<h2>Install and Import Required Libraries</h2>
    <p style="font-size: 16px">Installing PyPDF2 to extract text from the books</p>
</div>

In [1]:
!pip install PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 6.2 MB/s eta 0:00:00


In [2]:
import PyPDF2
from kaggle_secrets import UserSecretsClient
import datetime
import time
import google.generativeai as genai
from google.generativeai import caching

<div style="font-family:verdana" id="configure-gemini">
<h2>Configure the gemini api key</h2>
    <p style="font-size:16px">Get free api key from 
        <a href="https://aistudio.google.com/apikey"> Google AI Studio</a>  
    </p>

<p style="font-size:16px">To Configure api key in kaggle environment by clicking on Add-Ons on top bar then go to secrets and give it in key value pair. Ex. GEMINI_API_KEY = "API-KEY"</p>
</div>

In [3]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()

GEMINI_API_KEY = user_secrets.get_secret("GEMINI_API_KEY")
genai.configure(api_key=GEMINI_API_KEY)

<div style="font-family:verdana" id="extract-text">
    <h2>Extracting text from the books</h2>
    <p style="font-size:15px"></p>
</div>

In [4]:

# File paths for the books
book_files = {
    "prisoner_of_azkaban": "/kaggle/input/bookspdf/Harry Potter And The Prisoner Of Azkaban.pdf",
    "order_of_phoenix": "/kaggle/input/bookspdf/Harry-Potter-and-the-Order-of-Phoenix.pdf"
}

def extract_from_pdf(book) :
    with open(book, "rb") as file:
        reader = PyPDF2.PdfReader(file)
        book_text = ""
        for page in reader.pages:
            book_text += page.extract_text()
    return book_text

prizoner_of_azkaban_book_text = extract_from_pdf(book_files['prisoner_of_azkaban'])
order_of_phoenix_book_text    = extract_from_pdf(book_files['order_of_phoenix'])

<div style="font-family:verdana" id="write-prompts">
    <h2>Writing required prompts</h2>
     <p style="font-size:16px">To get more control on the response we will get, we need to write detailed prompts. We are splitting this prompts into <b style="color:blue">subparts of changes, story and response</b>. Later on we will merge it to give to model.</p>
    <p style="font-size:16px"> <b style="color:blue">change_extents --- </b>. In this we will keep the level of alteration we want in the story by adding the James Bond Character. It can be small extent, medium extent, drastic extent. We will have description for that to feed in the model so model can understand how much alteration is required and will produce output according to that.</p>
     <p style="font-size:16px"> <b style="color:blue">story_alteration_prizoner_of_azkaban ---</b> In this we will introduce James Bond first time in series and also give Bond's attributes, attitude, acheivements, success and other details about him, like how we want him to be in the story of Harry Potter and Prizoner of Azkaban. We are also defining few plots like what we want from James Bond in the story</b></p>

   <p style="font-size:16px"> <b style="color:blue">story_alteration_for_order_of_phoenix --- </b> In this we are reintroducing James Bond and similarly giving details about him and also definign some plots, like what we want from Bond in the story of Harry Potter and Order of Phoenix.</p> 
    <p style="font-size:16px"> <b style="color:blue">output_required --- </b> This is the list which is holding the final responses which we want. Like, we want dialogues between James Bond and other character after story alteration, we want summary after story alteration, we want script for 1 hour long movie and much more it can hold.</p>
</div>

In [5]:
change_extents = [
    {
        "extent": "Small Extent",
        "description": """
        In this case, the changes are very subtle and minimally impact the overall story. The core narrative of the book remains largely unchanged, but small adjustments are made to certain events or details. 
        These changes might include adding a minor character who appears only in a few scenes, altering the background or environment slightly, or introducing a piece of dialogue that doesn’t significantly shift the plot. 
        For instance, a character might make a brief appearance in a scene but have no major impact on the direction of the plot. Another example could be changing a minor decision made by a character, like a different choice for a meal or a location, without affecting the larger story arc. 
        The changes should not affect the primary goals or the structure of the narrative. The focus is on adding depth or nuance without altering the fundamental flow or emotional impact of the story.
        """
    },
    {
        "extent": "Medium Extent",
        "description": """
        Here, the changes are more noticeable and start to affect the progression of the story, but they do not completely rewrite the plot. Major events still unfold, and the overall storyline remains the same, but there are moderate alterations to how things occur or which characters are central to specific moments. 
        For example, a character who had a smaller role in the original narrative might now play a more significant part, like Hermione taking on a leadership role during a pivotal scene. 
        Alternatively, a scene or event might be altered so that the outcome is different, like a character choosing a different path or strategy in an important situation (e.g., Harry making a different choice during the Triwizard Tournament). 
        While the larger events and themes are maintained, the reader would notice the subtle but impactful difference in how the story unfolds due to the changes. These changes might affect character motivations or the dynamics between characters but do not drastically alter the central theme or outcome of the story.
        """
    },
    {
        "extent": "Drastic Extent",
        "description": """
        These changes are transformative and can significantly alter the course of the entire story. The plot may remain loosely familiar, but the events and character arcs could be reshaped in fundamental ways. 
        A new character, such as Sherlock Holmes, might be introduced and could have a major influence on key events in the narrative, shifting the balance of power or creating entirely new dynamics between characters. 
        For example, if Sherlock Holmes were inserted into the Harry Potter universe, he might help Harry solve mysteries or uncover clues about the Horcruxes in ways that completely change the original flow of the story. 
        The introduction of this new character could lead to completely different strategies or decisions in the plot, making the story unfold differently, but with a clear connection to the original themes and arcs. 
        These changes might also involve altering key events, such as replacing a character's role or deciding that a significant event happens in a different way (e.g., Voldemort's defeat occurring differently). 
        The core of the story could still be recognizable, but the transformation would be substantial, leading to a novel retelling that deviates from the original yet retains its essence.
        """
    }
]

In [6]:
story_alteration_prizoner_of_azkaban =  """ You are tasked with making changes to the story of *Harry Potter and the Prisoner of Azkaban*. In this version, James Bond, a British secret agent, plays a crucial role in the investigation surrounding Sirius Black’s escape from Azkaban. His presence significantly alters certain aspects of the plot while still maintaining the essence of the original story.
James Bond has the following characteristics and background:
- **Skills**: A master of espionage, expert in combat, skilled in using firearms, and highly tactical.
- **Achievements**: Known for defeating dangerous adversaries, such as the villain Le Chiffre in *Casino Royale* and stopping a terror plot in *Skyfall*.
- **Best Cases**: 
  - He infiltrated a dangerous terrorist organization in *Casino Royale*.
  - In *Skyfall*, he stopped an ex-agent from exposing MI6 secrets and launched a successful mission against a global threat.
  - In *Goldfinger*, he thwarted a villain’s plan to contaminate the U.S. gold reserves.

His involvement in the story should reshape key moments. For example:
- James Bond may assist Harry in tracking Sirius Black, using his advanced surveillance equipment to uncover crucial clues about Sirius’ escape.
- Bond could be directly involved in the investigation, offering insight into the operations of the Ministry of Magic or assisting Harry during moments of danger, such as when he faces the Dementors.
- In an important scene, Bond might confront a dark wizard or assist in an action-packed sequence, further shifting the storyline and giving Harry and his friends access to resources they wouldn't normally have.

The core narrative should still focus on Harry's journey and the mystery of Sirius Black, but James Bond’s influence should become more pronounced. He should provide actionable intelligence, use his espionage skills to uncover hidden truths, and even engage in combat situations to help Harry and his friend 
""" 


story_alteration_for_order_of_phoenix =  """
Now james bond is back in Harry Potter and Order of Phoenix. Now change the story of Harry Potter and 
Order of Phoenix Book to incoporate Bond. 
- **Skills**: A master of espionage, expert in combat, skilled in using firearms, and highly tactical.
- **Achievements**: Known for defeating dangerous adversaries, such as the villain Le Chiffre in *Casino Royale* and stopping a terror plot in *Skyfall*.
- **Best Cases**: 
  - He infiltrated a dangerous terrorist organization in *Casino Royale*.
  - In *Skyfall*, he stopped an ex-agent from exposing MI6 secrets and launched a successful mission against a global threat.
  - In *Goldfinger*, he thwarted a villain’s plan to contaminate the U.S. gold reserves.

His involvement in the story should reshape key moments. For example:
- James Bond may assist Harry in tracking Sirius Black, using his advanced surveillance equipment to uncover crucial clues about Sirius’ escape.
- Bond could be directly involved in the investigation, offering insight into the operations of the Ministry of Magic or assisting Harry during moments of danger, such as when he faces the Dementors.
- In an important scene, Bond might confront a dark wizard or assist in an action-packed sequence, further shifting the storyline and giving Harry and his friends access to resources they wouldn't normally have.
- But in the end James Bond get caught and ministry got to know that he is a spy of Muggles and sentence to 
  imprisonment in Azkaban but Dumbelodore and Harry save him
"""

In [7]:
output_required = [
    """
    ### Output Required ----- Give the scenarios in which you made the changes in book to accomodate James Bond and
    how the storyline got affected
    """,
    """
    ### Output required ----- Give all the dialogues between James Bond and other characters with including short description
    of where those dialogues happening
    """,
    """
    ### Output Required ----- 
    Create a detailed script for a 1-hour movie adaptation of the rewritten book with James Bond as a central character. Outline the key scenes 
    and plot points that will be featured in the film, making sure that the script retains the essence of the original story while adapting it for the screen. Break the script into acts, ensuring there is a clear beginning, middle, and climax. Include both the dialogues and actions in detail for each scene, providing descriptions of the setting, character emotions, and key visual elements. Pay special attention to pacing, dramatic moments, and how Bond’s character is portrayed through his actions and interactions with others. The script should be structured in a way that aligns with the conventions of a 1-hour action movie, including a compelling opening, rising tension, and a satisfying resolution.
    """,
  
    """
    ### Output Required:
    Provide a detailed summary of the altered version of *Harry Potter and the Prisoner of Azkaban*, focusing on:
    - The main plot and character developments.
    - How James Bond’s integration reshaped the narrative.
    - Key differences from the original story and their significance.

    """
]

<div style="font-family:Verdana" id="create-context-caching">
    <h2>Creating Context Caching</h2>
</div>

In [8]:


# Our use case is creative and need diverse output
# so we need to increase temperature as default is 1
# which will help us for getting creative and diverse output
generation_config = {
    "temperature": 1.5
}

# Model which we are using to leverage Long Context Window and Context Caching benefits
model = 'models/gemini-1.5-flash-002'

# System Instruction to the Model
# We are defining what specifically we want from the model
system_instruction = """
    You are great in understanding stories in the books and capturing all the information, relations and much more 
    from the book. You are great in alteration of the story by adding different characters or giving different roles to different characters where needed
    and changing the   enario an changing the story in book by the extent to which the user provide in the prompt. You also write text of book 
    in be fromat. Understand what user wants, whether to rewrte complete book after alterations or give summaries of new created book 
    or generate dialogues for 1 hour movie. Please provide output in proper and readable format. Do not be ambiguous and be to the point
    and give what is required
    """


In [9]:
# Create function for context caching

def caching_the_content(model, display_name, system_instruction, text_content) :
     
    cached_content = caching.CachedContent.create(
    model=model,
    display_name= display_name, 
    system_instruction= system_instruction,
    contents=[text_content],
    ttl=datetime.timedelta(minutes=20),
    )
    return cached_content

<div style="font-family:Verdana" id="prizoner-of-azkaban">
    <h2>Creativity on Harry Potter and Prizoner of Azkaban</h2>
</div>

In [10]:
cached_content_1 = caching_the_content(model, "Prizoner of Azkaban", system_instruction, prizoner_of_azkaban_book_text)

In [11]:
model_1 = genai.GenerativeModel.from_cached_content(cached_content=cached_content_1, generation_config=generation_config)

In [12]:
# Prompt 1

prompt_prizoner_of_azkaban_1 = story_alteration_prizoner_of_azkaban + "\n" +change_extents[1]['description'] + "\n" + output_required[0]
response_prizoner_of_azkaban_1 = model_1.generate_content(prompt_prizoner_of_azkaban_1)
print(response_prizoner_of_azkaban_1.text)

To incorporate James Bond into *Harry Potter and the Prisoner of Azkaban* without completely rewriting the plot, we can strategically insert him into existing scenarios, altering their dynamics and outcomes subtly but significantly.  Here are some scenarios and how Bond's involvement affects the storyline:

**1. Sirius Black's Escape and Initial Investigation:**

* **Original:** The Ministry's investigation into Black's escape is portrayed as somewhat haphazard and bureaucratic.
* **Change:**  The Ministry, realizing the gravity of the situation, secretly enlists 007's expertise.  Bond, using his advanced technological skills (miniature surveillance devices, satellite imagery, etc.), helps uncover overlooked clues regarding the escape – perhaps discovering a hidden passageway or a compromised guard. This speeds up the investigation and provides Harry with more immediate and accurate information about Black, potentially preventing some of the early misunderstandings and threats.  The st

In [13]:
# Prompt 2

prompt_prizoner_of_azkaban_2 = story_alteration_prizoner_of_azkaban + "\n" +change_extents[1]['description'] + "\n" + output_required[1]
response_prizoner_of_azkaban_2 = model_1.generate_content(prompt_prizoner_of_azkaban_2)
print(response_prizoner_of_azkaban_2.text)

Here are some key dialogues between James Bond and other characters in an altered *Harry Potter and the Prisoner of Azkaban* storyline, incorporating Bond's espionage expertise:


**Dialogue 1:**

* **Setting:** The Leaky Cauldron, immediately after Harry is unexpectedly rescued by the Knight Bus and brought to the attention of Minister Fudge. Bond, having been discreetly briefed by M (who has connections within the wizarding world), is already there.


* **Characters:** James Bond, Harry Potter, Cornelius Fudge.


* **Dialogue:**

> **Fudge:** (Staring at Harry) This is Harry Potter.
>
> **Bond:** (Calmly, assessing Harry) A pleasure, Mr. Potter. Minister Fudge, I understand Sirius Black's escape is causing some… *disruption*.  MI6 has some resources that might prove useful in this matter.  Perhaps a discreet conversation?

> **Fudge:** (Suspicious but intrigued)  MI6?  Very well, Mr. Bond. Let's talk inside.  


**Dialogue 2:**

* **Setting:** A private room above the Leaky Cauldron.

In [14]:
# Prompt 3

prompt_prizoner_of_azkaban_3 = story_alteration_prizoner_of_azkaban + "\n" +change_extents[1]['description'] + "\n" + output_required[2]
response_prizoner_of_azkaban_3 = model_1.generate_content(prompt_prizoner_of_azkaban_3)
print(response_prizoner_of_azkaban_3.text)

## Harry Potter and the Prisoner of Azkaban: 007 Intervention

**Logline:** When Sirius Black escapes Azkaban, Harry Potter isn't alone.  MI6's James Bond, alerted to a magical threat, joins the investigation, bringing his unique skills and resources to bear on a mystery that intertwines the wizarding and mundane worlds.


**Running Time:** 60 minutes


**Act I: The Escape (0-15 minutes)**

**Scene 1:** Opens with a thrilling montage of Sirius Black’s escape from Azkaban, intercut with news reports highlighting the escaped convict.  The tone is dark and ominous. We see glimpses of Azkaban’s chilling atmosphere, punctuated by Black’s determined movements.

**Scene 2:**  M16 Headquarters, London. James Bond (in a tailored suit, naturally) reviews intelligence reports on the escape, highlighting unusual magical disturbances detected near Azkaban.  M (Bond’s superior) explains the involvement of a powerful, ancient dark magic, prompting Bond's interest.  He receives a discreet encrypted me

In [15]:
# Prompt 4

prompt_prizoner_of_azkaban_4 = story_alteration_prizoner_of_azkaban + "\n" +change_extents[1]['description'] + "\n" + output_required[3]
response_prizoner_of_azkaban_4 = model_1.generate_content(prompt_prizoner_of_azkaban_4)
print(response_prizoner_of_azkaban_4.text)

## Harry Potter and the Prisoner of Azkaban: 007's Intervention

This revised version of *Harry Potter and the Prisoner of Azkaban* introduces James Bond as a key player in the investigation surrounding Sirius Black’s escape and subsequent pursuit of Harry.  The core mystery remains, but Bond's espionage expertise significantly alters the pace and methods of the investigation.

**Main Plot and Character Developments:**

The story begins similarly, with Harry discovering Black's escape from the news. However, unbeknownst to Harry,  M16, alerted by a tip-off regarding unusual magical activity near Azkaban, sends James Bond to investigate.  Bond, disguised as a Ministry of Magic consultant, discreetly observes the initial investigation, recognizing the unconventional nature of the escape and Black’s magical abilities.  He quietly assesses the Ministry's methods, identifying weaknesses and potential leads they've overlooked.

Bond’s high-tech surveillance equipment becomes crucial in the s

In [16]:
# Display Cached Contents
for c in caching.CachedContent.list():
  print(c)

CachedContent(
    name='cachedContents/kwym5rgnr9ui',
    model='models/gemini-1.5-flash-002',
    display_name='Prizoner of Azkaban',
    usage_metadata={
        'total_token_count': 150524,
    },
    create_time=2024-11-30 13:04:33.500015+00:00,
    update_time=2024-11-30 13:04:33.500015+00:00,
    expire_time=2024-11-30 13:24:33.120340+00:00
)


<div style="font-family:Verdana" id="delete-cache-1">
    <h3>Delete cached stored for prizoner of azkaban</h3>
    <p style="font-size:  16px"> We do not require cached stored for prizoner of azkaban now so we can delete it and then will create new cache for Order of Phoenix. This is required because, for gemini-1.5-flash model we have 1 million context caching limit, if we not delete this then unnecessary will occupy the space in conext cache and also will take cache storage cost</p>
</div>

In [17]:
# Delete cached content craeted for Prizoner of Azkaban to use it for Order of Phoenix
cached_content_1.delete()

<div style="font-family:Verdana" id="bond-return">
    <h2>James Bond return in Order of Phoenix</h2>
    <p style="font-size:16px">Now we have summary of altered story in Harry Potter and Prizoner of Azkaban. We will give it as a input conext tokens to the model along with the complete  Order of phoenix book to make it consistent in the series. Also we want responses to get consistent from the past and not generate much random and different things.</p>
</div>

In [18]:
summary_of_prizoner_of_azkaban = """ I am giving you the summary of the altered story of Harry Potter and Prizoner of Azkaban
by incoporating James Bond character. This is as follows - 
""" + prompt_prizoner_of_azkaban_4

In [19]:
cached_content_2 = caching_the_content(model, "Order of Phoenix", system_instruction, order_of_phoenix_book_text)

In [20]:
model_2 = genai.GenerativeModel.from_cached_content(cached_content=cached_content_2,generation_config=generation_config)

In [21]:
# Prompt 1

prompt_order_of_phoenix_1 = summary_of_prizoner_of_azkaban + "   \n" + story_alteration_for_order_of_phoenix  + "\n" + output_required[0]
response_order_of_phoenix_1= model_2.generate_content(prompt_order_of_phoenix_1)
print(response_order_of_phoenix_1.text)

Here's how the storyline of *Harry Potter and the Order of the Phoenix* could be altered to incorporate James Bond, focusing on the scenarios of change and their impact on the plot:

**Scenarios of Change and Storyline Impacts:**

1. **The Ministry Leak:** The book opens with Harry brooding at Privet Drive.  Instead of relying solely on overheard news snippets and vague letters, Bond, secretly tasked by Dumbledore (perhaps through a discreet contact within MI6), provides Harry with encrypted information about the Ministry's internal struggles and Voldemort's activities.  This information comes via a sophisticated communication device disguised as a seemingly ordinary item (a lighter, a pen, etc.) that Bond secretly plants on Harry during an unseen, nighttime encounter at Privet Drive.  The intel gives Harry a more proactive role early on, and introduces a level of suspense surrounding Bond’s involvement and the risk of exposure.


2. **Tracking Voldemort's Network:** Bond’s expertise i

In [22]:
#Prompt 2

prompt_order_of_phoenix_2 = summary_of_prizoner_of_azkaban + "   \n" + story_alteration_for_order_of_phoenix  + "\n" + output_required[1]
response_order_of_phoenix_2= model_2.generate_content(prompt_order_of_phoenix_2)
print(response_order_of_phoenix_2.text)

I cannot provide a complete rewrite of the *Harry Potter and the Order of the Phoenix* incorporating James Bond and focusing solely on their dialogues.  The task is excessively long and complex.  Generating a realistic and consistent narrative that blends two vastly different fictional universes, while maintaining the integrity of both source materials and incorporating detailed dialogue for every scene, is beyond the current capabilities of large language models.

However, I can offer a *sample* of potential dialogues to illustrate how such an integration *might* be done.  Remember this is a *fragment* and not a full story summary.


**Scene:**  The Burrow.  Harry, Ron, and Hermione are discussing the increasing attacks and Voldemort's resurgence.  James Bond, having been secretly contacted by Dumbledore, arrives discreetly.

**(Dialogue 1)**

**Hermione:**  ...and the Ministry still refuses to believe Voldemort's back!  It's infuriating!

**Ron:**  They're blind as bats!  Don't they 

In [23]:
#Prompt 3

prompt_order_of_phoenix_3 = summary_of_prizoner_of_azkaban + "   \n" + story_alteration_for_order_of_phoenix  + "\n" + output_required[2]
response_order_of_phoenix_3= model_2.generate_content(prompt_order_of_phoenix_3)
print(response_order_of_phoenix_3.text)

## Harry Potter and the Order of the Phoenix: 007 Intervenes (1-Hour Movie Script)

**Logline:**  When Voldemort's resurgence throws the wizarding world into chaos, Harry Potter receives unexpected aid from James Bond, a secret agent with unique skills and resources, but their alliance puts them both in grave danger.

**Characters:**

* Harry Potter
* Hermione Granger
* Ron Weasley
* James Bond
* Albus Dumbledore
* Dolores Umbridge
* Cornelius Fudge
* Voldemort
* Sirius Black (appears in flashbacks/visions)


**Act I: A Troubled Summer (approx. 15 minutes)**

**Scene 1:**  Privet Drive, Number Four. EXT. DAY.  Hot summer day. Harry is despondent, hidden behind a bush, listening to Muggle news, hoping for any sign of magical activity.  He's gaunt and tired.

**(Dialogue)**

* **Harry (V.O.):** Another day, another dead end.  Voldemort’s back, and I’m stuck here, listening to Muggle drivel...

**Scene 2:** Privet Drive, Number Four. INT. LIVING ROOM. NIGHT. The Dursleys are watching TV; 

In [24]:
for c in caching.CachedContent.list():
  print(c)

CachedContent(
    name='cachedContents/jg2jfn0nozjj',
    model='models/gemini-1.5-flash-002',
    display_name='Order of Phoenix',
    usage_metadata={
        'total_token_count': 398220,
    },
    create_time=2024-11-30 13:05:35.653791+00:00,
    update_time=2024-11-30 13:05:35.653791+00:00,
    expire_time=2024-11-30 13:25:35.298139+00:00
)


<div style="font-family:Verdana" id="delete-cache-2">
    <h3>Delete cached stored for order of phoenix</h3>
</div>

In [25]:
cached_content_2.delete()

<div style="font-family: Verdana" id="conclusion">
    <h2>Conclusion</h2>
    <p style="font-size:16px">
        In above use case we have demonstarted the beneficial use of Gemini 1.5 flash model long context window and optimization using context caching. We processed more than 150k input tokens in 3rd edition and more than 350k input tokens in 5th edition. We also used better prompt engineering techniques to give quality prompts to models to 
get high quality outputs. We also showed how context caching is benefitting us in the use case as we have to give multiple prompts on the same input context tokens.    </p>
      <p style="font-size:16px">
          We also showed how we can get character continuity in the use case by adding previous book's summary to the next book, so the storyline we get is not ambiguous. We deleted cached after our use to minimize the cost.
      </p>
</div>